In [ ]:
# Import libraries
from dotenv import load_dotenv
import os
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Load environment variables
load_dotenv()
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')

In [ ]:
# Connect to the Stack Overflow database
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)

In [ ]:
# Define HEART metrics for Stack Overflow
heart_metrics = {
    'Happiness': 'Measure of user satisfaction and happiness with the Questions & Answers product.',
    'Engagement': 'Measure of user interaction and activity on the platform.',
    'Adoption': 'Measure of user acquisition and growth of Stack Overflow user base.',
    'Retention': 'Measure of user retention and continued usage of the platform over time.',
    'Task Success': 'Measure of user ability to accomplish their goals and tasks effectively on Stack Overflow.'
}

When moving through the HEART framework's "Goals Signals Metrics" process, it can be useful to lay out these things out in a table like the following:

|  | Goals | Signals | Metrics |
| --- | --- | --- | --- |
| Happiness | Users find the product helpful, fun and easy to use | This could be signalled by users recommending Stack Overflow Questions & Answers to others.  | The score number being above a 164. |
| Engagement | Users enjoy product content and keep engaging with it | This could be signalled by the amount of time spent on the site, or the number of interactions with the content (comments, posts, voting, sharing). | total total interactions with the content (sum of comments, posts, shares, votes) |
| Adoption | New users see the value in the product | This could be signalled by the number of new user registrations. | total number of new user registrations  |
| Retention | Users keep coming back to the app to complete a key action | This could be signalled by the user retention rate over time. | percentage of users retained over specific time periods. |
| Task Success | Users complete their goal quickly and easily | This could be signalled by task success rate. | percentage of users successfully completing their tasks |

# Build SQL Queries

Write SQL queries to calculate the metrics

In [ ]:
# Happiness

with conn.cursor() as cursor:
    cursor.execute("SELECT COUNT(*) AS good_post_scores FROM posts WHERE score > 164;")
    df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

print(df)

In [ ]:
# Adoption

with conn.cursor() as cursor:
    cursor.execute("""
        WITH current_year_data AS (
            SELECT COUNT(*) AS new_user_registrations 
            FROM users 
            WHERE EXTRACT(YEAR FROM creation_date) = EXTRACT(YEAR FROM CURRENT_DATE)
        ),
        previous_year_data AS (
            SELECT COUNT(*) AS new_user_registrations 
            FROM users 
            WHERE EXTRACT(YEAR FROM creation_date) = EXTRACT(YEAR FROM CURRENT_DATE) - 1
            AND EXTRACT(MONTH FROM creation_date) BETWEEN 1 AND 6
        ) 
        SELECT 
            (SELECT new_user_registrations FROM current_year_data) AS current_year_registrations, 
            (SELECT new_user_registrations FROM previous_year_data) AS previous_year_registrations;
    """)
    result = cursor.fetchall()
df = pd.DataFrame(result, columns=['Current Year Registrations (Jan-June)', 'Previous Year Registrations (Jan-June)'])
display(df)

In [ ]:
with conn.cursor() as cursor:
    cursor.execute("""
        WITH user_counts AS (
            SELECT
                DATE_TRUNC('month', creation_date) AS account_creation_month,
                COUNT(*) AS num_new_users
            FROM users
            GROUP BY account_creation_month
        ),
        retained_users AS (
            SELECT
                DATE_TRUNC('month', u.creation_date) AS account_creation_month,
                COUNT(*) AS num_retained_users
            FROM users u
            WHERE EXISTS (
                SELECT 1
                FROM users u2
                WHERE u2.account_id = u.account_id
                AND DATE_TRUNC('month', u2.last_access_date) >= DATE_TRUNC('month', u.creation_date) + INTERVAL '1 month'
                AND DATE_TRUNC('month', u2.last_access_date) <= DATE_TRUNC('month', u.creation_date) + INTERVAL '6 month'
            )
            GROUP BY account_creation_month
        )
        SELECT
            uc.account_creation_month,
            uc.num_new_users,
            COALESCE(num_retained_users, 0) AS num_retained_users
        FROM user_counts uc
        LEFT JOIN retained_users ru ON uc.account_creation_month = ru.account_creation_month
        ORDER BY uc.account_creation_month;
    """)

    # Fetch the result
    result = cursor.fetchall()

# Create a DataFrame from the query result
df = pd.DataFrame(result, columns=['Account Creation Month', 'New Users', 'Retained Users'])

# Display the DataFrame as a table in the Jupyter notebook
display(df)

In [ ]:
with conn.cursor() as cursor:
    cursor.execute("""
        SELECT
            DATE_TRUNC('month', p.creation_date) AS "Month",
            COUNT(DISTINCT CASE WHEN p.accepted_answer_id IS NOT NULL THEN p.id END) AS "Accepted Answers",
            COUNT(DISTINCT CASE WHEN p.post_type_id = 1 THEN p.id END) AS "Total Questions",
            COUNT(DISTINCT CASE WHEN p.accepted_answer_id IS NOT NULL THEN p.id END) /
                NULLIF(COUNT(DISTINCT CASE WHEN p.post_type_id = 1 THEN p.id END), 0) AS "Task Success Rate"
        FROM posts p
        WHERE p.creation_date >= DATE_TRUNC('year', CURRENT_DATE) - INTERVAL '2 year'
        GROUP BY DATE_TRUNC('month', p.creation_date)
        ORDER BY DATE_TRUNC('month', p.creation_date);
    """)

    # Fetch the result
    result = cursor.fetchall()

# Create a DataFrame from the query result
df = pd.DataFrame(result, columns=['Month', 'Accepted Answers', 'Total Questions', 'Task Success Rate'])

# Display the DataFrame as a table in the Jupyter notebook
display(df)

The above indicates that there are 84,548 posts with a score above 164. Having a score above 164 is considered good. The score of a post is calculated by:
   * number of upvotes - number of downvotes


In [ ]:
# Define SQL queries
sql_queries = {
    'Happiness': "SELECT COUNT(*) AS good_post_scores FROM posts WHERE score > 164;",
    'Engagement': "SELECT SUM(comment_count) AS total_comments FROM posts;",
    'Adoption': """
        WITH current_year_data AS (
            SELECT COUNT(*) AS new_user_registrations 
            FROM users 
            WHERE EXTRACT(YEAR FROM creation_date) = EXTRACT(YEAR FROM CURRENT_DATE)
        ),
        previous_year_data AS (
            SELECT COUNT(*) AS new_user_registrations 
            FROM users 
            WHERE EXTRACT(YEAR FROM creation_date) = EXTRACT(YEAR FROM CURRENT_DATE) - 1
            AND EXTRACT(MONTH FROM creation_date) BETWEEN 1 AND 6
        ) 
        SELECT 
            (SELECT new_user_registrations FROM current_year_data) AS current_year_registrations, 
            (SELECT new_user_registrations FROM previous_year_data) AS previous_year_registrations;
    """,
    'Retention': """
        WITH user_counts AS (
            SELECT
                DATE_TRUNC('month', creation_date) AS account_creation_month,
                COUNT(*) AS num_new_users
            FROM users
            GROUP BY account_creation_month
        ),
        retained_users AS (
            SELECT
                DATE_TRUNC('month', u.creation_date) AS account_creation_month,
                COUNT(*) AS num_retained_users
            FROM users u
            WHERE EXISTS (
                SELECT 1
                FROM users u2
                WHERE u2.account_id = u.account_id
                AND DATE_TRUNC('month', u2.last_access_date) >= DATE_TRUNC('month', u.creation_date) + INTERVAL '1 month'
                AND DATE_TRUNC('month', u2.last_access_date) <= DATE_TRUNC('month', u.creation_date) + INTERVAL '6 month'
            )
            GROUP BY account_creation_month
        )
        SELECT
            uc.account_creation_month,
            uc.num_new_users,
            COALESCE(num_retained_users, 0) AS num_retained_users
        FROM user_counts uc
        LEFT JOIN retained_users ru ON uc.account_creation_month = ru.account_creation_month
        ORDER BY uc.account_creation_month;
    """,
    'Task_Success': """
        SELECT
            DATE_TRUNC('month', p.creation_date) AS "Month",
            COUNT(DISTINCT CASE WHEN p.accepted_answer_id IS NOT NULL THEN p.id END) AS "Accepted Answers",
            COUNT(DISTINCT CASE WHEN p.post_type_id = 1 THEN p.id END) AS "Total Questions",
            COUNT(DISTINCT CASE WHEN p.accepted_answer_id IS NOT NULL THEN p.id END) /
                NULLIF(COUNT(DISTINCT CASE WHEN p.post_type_id = 1 THEN p.id END), 0) AS "Task Success Rate"
        FROM posts p
        WHERE p.creation_date >= DATE_TRUNC('year', CURRENT_DATE) - INTERVAL '2 year'
        GROUP BY DATE_TRUNC('month', p.creation_date)
        ORDER BY DATE_TRUNC('month', p.creation_date);
    """
}

# Execute SQL queries and display results as tables
for metric, query in sql_queries.items():
    df = pd.read_sql_query(query, conn)
    print(f"--- {metric} ---")
    print(df)
    print("\n")